In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-12-11"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
12041,2024-12-11,Eua Copa Nba,21:00,New York Knicks,Atlanta Hawks,1.37,3.28,235.5,1.88,1.96,-7.5,2.04,1.81,86The1Qj,0.729927,0.304878,0.531915,0.510204,0.034805,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.580892,0.029463,0.084485,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
12042,2024-12-11,Eua Copa Nba,23:30,Houston Rockets,Golden State Warriors,1.67,2.29,221.5,1.85,2.00,-3.5,2.00,1.84,8K8R90up,0.598802,0.436681,0.540541,0.500000,0.035484,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.221417,0.055099,0.058926,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
12043,2024-12-11,Europa Eurocopa,13:30,BC Wolves,Hapoel Tel-Aviv,2.76,1.41,170.5,1.87,1.79,5.5,1.91,1.80,I5GeTqZr,0.362319,0.709220,0.534759,0.558659,0.071539,0.8,0.2,NaN,NaN,205.288,66.179710,0.322375,101.496,13.745557,0.135430,131.938,103.856,129.36,88.33,0.0,0.0,0.0,0.0,0.457839,0.030912,0.041931,-0.52,-0.104,-16.923077,0.553638,0.4,-0.153638,0.04,0.008,51.250000,0.735643,0.6,-0.135643
12044,2024-12-11,Europa Eurocopa,13:30,Turk Telekom,Aris,1.24,3.77,156.5,1.83,1.83,-10.5,1.95,1.76,Ei0bm5QE,0.806452,0.265252,0.546448,0.546448,0.071704,0.6,0.8,NaN,NaN,149.696,48.567372,0.324440,333.232,175.211399,0.525794,145.348,325.448,141.68,401.32,0.0,0.0,0.0,0.0,0.714164,0.000000,0.072426,-0.75,-0.150,-1.600000,0.622208,0.6,-0.022208,3.33,0.666,4.159159,0.299599,0.4,0.100401
12045,2024-12-11,Europa Eurocopa,14:00,Cluj-Napoca,JL Bourg,1.75,1.98,174.5,1.84,1.82,-2.5,2.01,1.72,4MvwtNIl,0.571429,0.505051,0.543478,0.549451,0.076479,0.8,0.6,NaN,NaN,149.004,42.235910,0.283455,140.268,63.131868,0.450080,164.492,133.904,99.18,160.16,0.0,0.0,0.0,0.0,0.087204,0.007728,0.109952,-1.05,-0.210,-3.571429,0.723696,0.7,-0.023696,-0.14,-0.028,-35.000000,0.636601,0.7,0.063399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12187,2024-12-11,Finlândia Korisliiga Feminina,13:30,Vimpelin Veto F,Espoo Team F,1.66,2.04,153.5,1.85,1.81,-5.5,1.95,1.72,WG8NcSEl,0.602410,0.490196,0.540541,0.552486,0.092606,0.4,0.4,NaN,NaN,272.380,191.254300,0.702160,244.050,165.037964,0.676247,202.934,561.340,454.79,182.28,1.0,0.0,0.0,0.0,0.145244,0.015456,0.088629,-3.58,-0.716,-0.921788,0.572535,0.5,-0.072535,5.56,1.112,0.935252,0.304835,0.4,0.095165
12188,2024-12-11,Lituânia King Mindaugas Cup,13:30,Siauliai,Nevezis,2.01,1.68,176.5,1.81,1.85,-5.5,2.01,1.68,fyHUKehd,0.497512,0.595238,0.552486,0.540541,0.092751,0.8,0.4,NaN,NaN,229.234,93.379359,0.407354,195.718,81.518491,0.416510,121.548,292.528,174.60,132.60,0.0,0.0,0.0,0.0,0.126474,0.015456,0.126474,-2.78,-0.556,-1.816547,0.000000,0.0,0.000000,5.96,1.192,0.570470,0.000000,0.0,0.000000
12189,2024-12-11,Lituânia King Mindaugas Cup,14:00,Mazeikiai,Jonava,2.01,1.68,178.5,1.85,1.81,1.5,1.88,1.78,6PNFrap3,0.497512,0.595238,0.540541,0.552486,0.092751,0.0,1.0,NaN,NaN,296.684,121.843705,0.410685,207.370,73.524514,0.354557,0.000,363.580,196.69,179.92,0.0,0.0,0.0,0.0,0.126474,0.015456,0.038640,0.00,0.000,inf,0.000000,0.0,0.000000,-3.27,-0.654,-1.039755,0.000000,0.0,0.000000
12190,2024-12-11,Noruega Blno Feminina,13:30,Asker/Bærums Verk F,Bærum F,1.11,5.17,119.5,1.83,1.83,-13.5,1.95,1.72,dhwgqHb4,0.900901,0.193424,0.546448,0.546448,0.

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,21:00,Eua Copa Nba,New York Knicks,Atlanta Hawks,235.5,1.88,0.8684,Over
1,23:30,Eua Copa Nba,Houston Rockets,Golden State Warriors,221.5,1.85,0.9237,Over
